<a href="https://colab.research.google.com/github/sahupra1357/Pawpularity/blob/main/PetFinderCNNs.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install -U -q kaggle==1.5.8

     |████████████████████████████████| 59 kB 2.6 MB/s 


In [ ]:
!ls

sample_data


In [ ]:
!mkdir -p ~/.kaggle

In [ ]:
from google.colab import files
files.upload()

In [ ]:
!cp kaggle.json ~/.kaggle/

In [ ]:
!ls

kaggle.json  sample_data


In [ ]:
!chmod 600 ~/.kaggle/kaggle.json

In [ ]:
!kaggle competitions download -c petfinder-pawpularity-score

 98% 961M/983M [00:33<00:00, 25.1MB/s]
100% 983M/983M [00:33<00:00, 31.0MB/s]


In [ ]:
!ls

kaggle.json  petfinder-pawpularity-score.zip  sample_data


In [ ]:
!unzip petfinder-pawpularity-score.zip

In [ ]:
!ls

kaggle.json			 sample_data		test	  train
petfinder-pawpularity-score.zip  sample_submission.csv	test.csv  train.csv


In [ ]:
!ls train |wc -l

9912


In [ ]:
!ls test |wc -l

8


In [ ]:
import pandas as pd

df_training = pd.read_csv("train.csv")
df_test = pd.read_csv("test.csv")
Ids = df_test["Id"]

df_training["Id"] = df_training["Id"].apply(lambda x :  x + ".jpg")
df_test["Id"] = df_test["Id"].apply(lambda x : x + ".jpg")

In [ ]:
df_training.head()

,Id,Subject Focus,Eyes,Face,Near,Action,Accessory,Group,Collage,Human,Occlusion,Info,Blur,Pawpularity
0,0007de18844b0dbbb5e1f607da0606e0.jpg,0,1,1,1,0,0,1,0,0,0,0,0,63
1,0009c66b9439883ba2750fb825e1d7db.jpg,0,1,1,0,0,0,0,0,0,0,0,0,42
2,0013fd999caf9a3efe1352ca1b0d937e.jpg,0,1,1,1,0,0,0,0,1,1,0,0,28
3,0018df346ac9c1d8413cfcc888ca8246.jpg,0,1,1,1,0,0,0,0,0,0,0,0,15
4,001dc955e10590d3ca4673f034feeef2.jpg,0,0,0,1,0,0,1,0,0,0,0,0,72


In [ ]:
TRAIN_PCT = 0.9
TRAIN_CUT = int(len(df_training) * TRAIN_PCT)

df_train = df_training[0:TRAIN_CUT]
df_validate = df_training[TRAIN_CUT:]

print(f"Training size: {len(df_train)}")
print(f"Validate size: {len(df_validate)}")

Training size: 8920
Validate size: 992


In [ ]:
import tensorflow as tf
import keras_preprocessing
from keras_preprocessing import image
from keras_preprocessing.image import ImageDataGenerator
import os

IMAGES_DIR = os.path.abspath("train"+os.sep)

In [ ]:
training_datagen = ImageDataGenerator(
  rescale = 1./255,
  horizontal_flip=True,
  vertical_flip=True,
  fill_mode='nearest')

train_generator = training_datagen.flow_from_dataframe(
        dataframe=df_train,
        directory=IMAGES_DIR,
        x_col="Id",
        y_col="Pawpularity",
        target_size=(256, 256),
        batch_size=32,
        class_mode='other')

Found 8920 validated image filenames.


In [ ]:
validation_datagen = ImageDataGenerator(rescale = 1./255)

val_generator = validation_datagen.flow_from_dataframe(
        dataframe=df_validate,
        directory=IMAGES_DIR,
        x_col="Id",
        y_col="Pawpularity",
        target_size=(256, 256),
        class_mode='other')

Found 992 validated image filenames.


In [ ]:
from tensorflow.keras.callbacks import EarlyStopping

model = tf.keras.models.Sequential([
    tf.keras.layers.Conv2D(64, (3,3), activation='relu', input_shape=(256, 256, 3)),
    tf.keras.layers.MaxPooling2D(2, 2),
    tf.keras.layers.Conv2D(64, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(512, activation='relu'),
    tf.keras.layers.Dense(1, activation='linear')
])

model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 254, 254, 64)      1792      
                                                                 
 max_pooling2d (MaxPooling2D  (None, 127, 127, 64)     0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 125, 125, 64)      36928     
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 62, 62, 64)       0         
 2D)                                                             
                                                                 
 flatten (Flatten)           (None, 246016)            0         
                                                                 
 dense (Dense)               (None, 512)               1

In [ ]:
model.compile(loss = 'mean_squared_error', optimizer='adam')
monitor = EarlyStopping(monitor='val_loss', min_delta=1e-3, patience=5, verbose=1, mode='auto',
        restore_best_weights=True)
history = model.fit(train_generator,  
  verbose = 1, 
  validation_data=val_generator, callbacks=[monitor], epochs=25)

Epoch 1/25
279/279 [==============================] - 172s 502ms/step - loss: 526.5123 - val_loss: 423.1399
Epoch 2/25
279/279 [==============================] - 138s 494ms/step - loss: 438.1878 - val_loss: 417.9255
Epoch 3/25
279/279 [==============================] - 137s 491ms/step - loss: 443.1782 - val_loss: 434.6803
Epoch 4/25
279/279 [==============================] - 137s 491ms/step - loss: 429.5083 - val_loss: 412.0766
Epoch 5/25
279/279 [==============================] - 136s 487ms/step - loss: 424.0412 - val_loss: 439.1227
Epoch 6/25
279/279 [==============================] - 137s 490ms/step - loss: 416.4709 - val_loss: 420.1570
Epoch 7/25
279/279 [==============================] - 137s 490ms/step - loss: 405.2761 - val_loss: 423.8831
Epoch 8/25
279/279 [==============================] - 137s 489ms/step - loss: 394.1414 - val_loss: 427.2176
Epoch 9/25
279/279 [==============================] - 137s 490ms/step - loss: 383.5020 - val_loss: 447.3080
Epoch 00009: early stopping


In [ ]:
IMAGES_DIR_TEST = os.path.abspath("test"+os.sep)

In [ ]:
test_datagen = ImageDataGenerator(rescale = 1./255)

test_generator = test_datagen.flow_from_dataframe(
        dataframe=df_test,
        directory=IMAGES_DIR_TEST,
        x_col="Id",
        batch_size=1,
        shuffle=False,
        target_size=(256, 256),
        class_mode=None)

Found 8 validated image filenames.


In [ ]:
test_generator.reset()
pred = model.predict(test_generator,steps=len(df_test))

In [ ]:
df_submit = pd.DataFrame({'Id':Ids,'Pawpularity':pred.flatten()})

In [ ]:
#df_submit.to_csv("submission.csv",index=False)

In [ ]:
df_submit